In [1]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
from collections import OrderedDict
import pylhe
import glob
import pyslha
import xml.etree.ElementTree as ET

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

# Set output and input files

In [2]:
inputDir = '../MadAnalysis5/axial_all_run_01/'

### Get hepMC (input) folder and extract model info

In [3]:
inputFile = os.path.join(inputDir,'Input/_defaultset.list')
with open(inputFile,'r') as f:
    hepMCfile = f.read()
    hepDir = os.path.dirname(hepMCfile)
print('HepMC input dir=',hepDir)

HepMC input dir= /home/lessa/MonoXSMS/DMSimp_axial_all/Events/run_01


#### Get Model Parameters

In [4]:
banner = sorted(glob.glob(hepDir+'/*banner.txt'),key=os.path.getmtime,reverse=True)
if len(banner) == 0:
    print('Banner not found for %s' %label)
elif len(banner) > 1:        
    print('\n%i banner files found for %s. Using %s' 
          %(len(banner),label,os.path.basename(banner[0])))
banner = banner[0]
xtree = ET.parse(banner)
xroot = xtree.getroot()
genInfo = xroot.find('header').find('MGGenerationInfo').text.strip().split('\n')
genInfo = [x.replace('#','').strip().split(':') for x in genInfo]
nevts = [eval(x[1]) for x in genInfo if 'Number of Events' in x[0]][0]
xsecPBall = [eval(x[1]) for x in genInfo if 'Integrated weight (pb)' in x[0]]
xsecPBmatched = [eval(x[1]) for x in genInfo if 'Matched Integrated weight (pb)' in x[0]]
if xsecPBmatched:
    nevts = nevts*(xsecPBmatched[0]/xsecPBall[0])
    xsecPB = xsecPBmatched[0]
else:
    xsecPB = xsecPBall[0]

slha = xroot.find('header').find('slha').text
pars = pyslha.readSLHA(slha)
mMed = pars.blocks['MASS'][55]
mDM = pars.blocks['MASS'][52]
gVq = pars.blocks['DMINPUTS'][4] # Mediator-quark vector coupling
gAq = pars.blocks['DMINPUTS'][10] # Mediator-quark axial coupling
gVx = pars.blocks['DMINPUTS'][2] # Mediator-DM vector coupling
gAx = pars.blocks['DMINPUTS'][3] # Mediator-DM axial coupling
print('Cross-section (pb) = %1.3e' %xsecPB)
print('Number of Events = %i' %nevts)
print('mMed = %1.2f GeV, mDM = %1.2f GeV, gVq = %1.2f, gAq = %1.2f, gVx = %1.2f, gAx = %1.2f' 
      %(mMed,mDM,gVq,gAq,gVx,gAx))

Cross-section (pb) = 3.907e-01
Number of Events = 64496
mMed = 2000.00 GeV, mDM = 1.00 GeV, gVq = 0.00, gAq = 0.25, gVx = 0.00, gAx = 1.00


In [5]:
dataDict = {}
dataDict['filename'] = hepMCfile
dataDict['Total xsec (pb)'] = xsecPB
dataDict['Total MC Events'] = nevts
if gVx != 0:
    dataDict['Coupling'] = 'Vector'
else:
    dataDict['Coupling'] = 'Axial'
    
dataDict['Mode'] = 'DM+QCDjets'

dataDict['$m_{med}$'] = mMed
dataDict['$m_{DM}$'] = mDM
if dataDict['Coupling'] == 'Vector':
    dataDict['$g_{DM}$'] = gVx
    dataDict['$g_{q}$'] = gVq
else:
    dataDict['$g_{DM}$'] = gAx
    dataDict['$g_{q}$'] = gAq


### Get efficiencies

In [6]:

effFile = os.path.join(inputDir,'Output/SAF/CLs_output_summary.dat')
with open(effFile,'r') as f:
    for il,l in enumerate(f.readlines()):
        if il == 0:
            l = l.replace('#','')
            header = [x.strip() for x in l.split('  ') if x.strip()]
            dataDict.update({col : [] for col in header})
        else:
            pt = [None]*len(dataDict)
            if not l.split():
                continue
            for ix,x in enumerate(l.split()):
                if not x.strip():
                    continue
                try:
                    x = eval(x)
                except:
                    pass
                if ix > len(dataDict)-1:
                    continue
                pt[ix] = x
            for icol,col in enumerate(header):
                dataDict[col].append(pt[icol])
if '||' in dataDict:
    dataDict.pop('||')

In [7]:
df = pd.DataFrame.from_dict(dataDict)

In [8]:
df

,filename,Total xsec (pb),Total MC Events,Coupling,Mode,$m_{med}$,$m_{DM}$,$g_{DM}$,$g_{q}$,dataset name,analysis name,signal region,best?,sig95(exp),sig95(obs),1-CLs,efficiency,stat
0,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2016_bin0,0,19.515813,23.217032,0.0196,0.002592,0.000025
1,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2016_bin1,0,13.026438,14.741203,0.0381,0.002102,0.000023
2,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2016_bin2,0,6.784303,7.039151,0.0823,0.001786,0.000021
3,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2016_bin3,0,5.460833,4.766895,0.1464,0.001407,0.000019
4,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2016_bin4,0,3.672409,4.447895,0.1028,0.001328,0.000018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2018_bin18,0,0.954824,1.842439,0.1038,0.000158,0.000006
63,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2018_bin19,0,1.283539,1.741554,0.2591,0.000095,0.000005
64,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2018_bin20,0,0.948726,1.409871,0.2980,0.000111,0.000005
65,/home/lessa/MonoXSMS/DMSimp_axial_all/Events/r...,0.390705,64496.33005,Axial,DM+QCDjets,2000.0,1.0,1.0,0.25,defaultset,cms_exo_20_004,monojet_SR_2018_bin21,0,1.017722,1.937302,0.0981,0.000127,0.000006


### Save DataFrame to pickle file

In [9]:
outputFile = os.path.join(inputDir,'summary.pcl')
print('Saving to',outputFile)
df.to_pickle(outputFile)

Saving to ../MadAnalysis5/axial_all_run_01/summary.pcl
